In [ ]:
header = """
   _____      __    _   __     __  .____ 
  / ___/_____/ /_  / | / /__  / /_./ __ \/ /__   ____ 
  \__ \/ ___/ __ \/  |/ / _ \/ __./ /_/ / //_/ / __  / 
 ___/ / /__/ / / / /|  /  __/ /_./ ____/  <   | /_/  \
/____/\___/_/ /_/_/ |_/\___/\__./_/   /_/|_|  \__,_/\_\   
"""

In [ ]:
from Bio.PDB import *
from Bio import PDB
from torch import nn
import numpy as np
from ase import Atoms, Atom
import dask.dataframe as dd
from ordered_set import OrderedSet
import os
from collections import Counter
from collections import defaultdict
from itertools import chain, product
import sys
import string
char_list = (string.ascii_uppercase)
protonatable_sites2 = {"G":"GLH", 
                      "C":"CYS", 
                      "L":"LYS", 
                      "A":"ASH",
                      "H": "HIP",
                      "T": "TYR",
                      "X": "CTR",
                      "N": "NTR"}
protonatable_sites = {"G":"HE2", 
                      "C":"HG", 
                      "L":"HZ1", 
                      "A":"HD2",
                      "H": ("HD1","HE2"), #this needs to be fixed 
                      "T": "HH",
                      "X": ("OXT", "C"),
                      "N": "N"}
protonatable_sites3 = {"GLU":"GLH", 
                      "ASP":"ASH",
                      "HIS": "HIP"}


def atoms_to_structure(cutout, filename): 
    """input: cutout [<atom>, <atom>]: list of biopython atom objects"""
    #file name has the fname, which hs the resnumber and 199lA108_G29
    tit_res_names = None
    resname=None
    Tresid =filename.split("_")
    f1=Tresid[0][4:] #A108
    T_IDn,T_IDc = [int(f1[1:])], [f1[0]] 

    if len(Tresid) > 3: #merged
        f2 = Tresid[2][4:]
        T_IDn.append(int(f2[1:]))
        T_IDc.append(f2[0])

    chain_dict, res_dict = {}, {}
    structure = Structure.Structure("")
    model = Model.Model(0)
    structure.add(model)

    for atom in cutout:
        resname=None
        res = atom.get_parent()
        id = res.get_full_id()
        res_id, chain_id = id[3], id[2]

        if res_id[1] in T_IDn: #acidify 
            if chain_id in T_IDc:
                resname = protonatable_sites3.get(res.get_resname())
                if not resname: #bc ter etc arent in dic
                    resname = res.get_resname()
            else:
                resname = res.get_resname()

        else:
            resname = res.get_resname()
       
#make/retrieve chain
        if chain_id not in chain_dict:
            chain = Chain.Chain(chain_id) #make new chain
            chain_dict[chain_id] = chain
            model.add(chain) #add it

        else:
            chain = chain_dict[chain_id]
        
#make/retrieve residue
        if res_id not in res_dict:
            residue = Residue.Residue(res_id, resname, '')
            res_dict[res_id] = residue
            chain.add(residue) #add it
        else:
            residue = res_dict[res_id]

        residue.add(atom)
#save
    io = PDBIO()
    io.set_structure(structure)
    io.save(f"cuts/{filename}.pdb")

def _get_cut_AspGlu(residue, counter, distance_cutoff):
    cuts=[]
    sites=protonatable_sites[resname]
    atom1,atom2=residue[sites[0]],residue[sites[1]]
    if atom1.is_disordered(): 
        center, resname = atom1.get_coord(), resname + ","
    elif atom2.is_disordered():
        center, resname = atom2.get_coord(), resname + ","
    else:
        center=(atom1.get_coord() + atom2.get_coord()) / 2.0
    cut = ns.search(center, distance_cutoff, "A") #put ns search i n below? todo
    cuts.append((counter, center, resname, cut)) 
    return cuts

def _get_cut_normal(residue, counter, distance_cutoff):
    site=residue[protonatable_sites[resname]]
    if site.is_disordered(): 
        resname = resname + ","
    center = site.get_coord()
    cut = ns.search(center, distance_cutoff, "A")
    cuts.append((counter, center, resname, cut))
    return cuts

def generate_cutout_around_protonatable_site(residue, distance_cutoff, ns, counter, resname):
    """Residue wise resolurion. ns is neighbor search set up for the entire protein, residue is the single data point / 1 of several residues in a pdb & in pypka.
    input is one residue. output is the cutout around its titratable site, both of which can be plural e.g. his, mb asp and glu.
    residue (biopython Residue object): a single protonable residue """
    protonatable_sites = {"G":("OE1","OE2"), "A":("OD1","OD2"), "C":"SG", "L":"NZ", "H":("NE2", "ND1"), "T":"OH"} #TODO
    cuts = []

    if resname==0: #NTR
        center = residue['N'].get_coord()
        cut = ns.search(center, distance_cutoff, "A")
        cuts.append((counter, center, 'N', (cut, residue))) 
        return cuts
    
    elif resname==1: #CTR
        try:
            center = residue['OXT'].get_coord()
            cut = ns.search(center, distance_cutoff, "A")
            cuts.append((counter, center, 'X', (cut, residue)))
        except:
            center = residue['C'].get_coord()
        cut = ns.search(center, distance_cutoff, "A")
        cuts.append((counter, center, 'X', (cut, residue)))
        return cuts
    
    else:
        if resname=="G": 
            return _get_cut_AspGlu(counter, distance_cutoff)
        if resname=="A": 
            return _get_cut_AspGlu(counter, distance_cutoff)
        if resname=="C": 
            return _get_cut_normal(residue, counter, distance_cutoff)
        if resname=="L": 
            return _get_cut_normal(residue, counter, distance_cutoff)
        if resname=="T":
            return _get_cut_normal(residue, counter, distance_cutoff)
        if resname=="H":
            sites=protonatable_sites[resname]
            atom1,atom2=residue[sites[0]],residue[sites[1]]            
            if atom1.is_disordered(): 
                resname = resname + ","
            if atom2.is_disordered():
                resname = resname + ","
            center1,center2=atom1.get_coord(), atom2.get_coord()
            cut1, cut2 = ns.search(center1, distance_cutoff, "A"), ns.search(center2, distance_cutoff, "A")
            cuts.append([(counter+.1, center1, resname, cut1),(counter+.2, center2, resname, cut2)])

    return cuts #plural because of sites with multiple sites.


def merge_or_not_cutouts(cutouts_apdb, distance_cutoff): #TODO: reduce dtypes #PDB WISE!
    """
    in: all of the cutouts from the pdb. returns  [the merged or solo cutout for each input residue of cutouts_apdb]. len in = len out"""
    #protein wise ..
    dp_ids,centers,cuts, Ds_lite, cutouts, resnames,redunant_merged_is, done_pairs, other_site =[],[],[],[], [],[],[],[],[]

    for site in cutouts_apdb:
        if type(site)==tuple: 
            dp_ids.append(site[0])
            centers.append(site[1]) 
            resnames.append(site[2])
            cuts.append(site[3])
            other_site.append(None) 

        else: #his is list
            dp_ids.append(site[0][0]) 
            dp_ids.append(site[1][0])

            centers.append(site[0][1]) 
            centers.append(site[1][1]) 

            resnames.append(site[0][2])
            resnames.append(site[1][2]) 

            cuts.append(site[0][3])
            cuts.append(site[1][3])

            other_site.append(site[1][1]) #TODO
            other_site.append(site[0][1])


    num_residues=len(centers)
    distances = np.zeros((num_residues, num_residues))
    
    for i in range(num_residues):
        for j in range(i + 1, num_residues):
            distance = np.linalg.norm(centers[i] - centers[j]).astype(np.float32)
            if distance < distance_cutoff:
                distances[i, j] = distance.astype(np.float32)
                distances[j, i] = distance.astype(np.float32)

    Ds_lite = [distances[i][distances[i] != 0] for i in range(num_residues)] #nonzero entries

    #residuewise...
    for i in range(len(Ds_lite)): #=len IDs 
        a_residues_distance_array=Ds_lite[i]
        a_residues_distance_array.sort() #debug dont forget i messed around in here #TODO

        if a_residues_distance_array.any(): #merged
            index=i
            closest_cutout_i = int(np.where((distances[int(index), :])==a_residues_distance_array[0])[0])
            cut1,cut2=cuts[i], cuts[closest_cutout_i]
            
            #exclude own his
            if resnames[i] == 'H' and resnames[closest_cutout_i] == 'H':
                if other_site[i].any() and other_site[closest_cutout_i].any(): 
                    if len(a_residues_distance_array) > 1:
                        closest_cutout_i = int(np.where((distances[int(index), :])==a_residues_distance_array[1])[0]) #next closest
                    else:
                        pass #to solo cutouts

            pair_i = frozenset((index,dp_ids[closest_cutout_i]))#key #frozen set is immutable thus can be used as a dict key #also order doesnt matter, 2-1=1-2
            
            if not done_pairs: #if there are any yet merged
                cut1,cut2=cuts[i], cuts[closest_cutout_i]
                
                if type(cut1) == tuple:
                    cutout = (list(set(cut1[0] + cut2)),resnames[i] + resnames[closest_cutout_i], (centers[i], centers[closest_cutout_i]), cut1[1])
                elif type(cut2) == tuple: #safe bc no combined terminus
                    cutout = (list(set(cut1 + cut2[0])),resnames[i] + resnames[closest_cutout_i], (centers[i], centers[closest_cutout_i]), cut2[1])
                else:
                    cutout = (list(set(cut1 + cut2)),resnames[i] + resnames[closest_cutout_i], (centers[i], centers[closest_cutout_i]))

                done_pairs.append(pair_i)
                redunant_merged_is.append(closest_cutout_i)

            else: #if there are already some generated
                if pair_i not in done_pairs: #if that mergedcut hasnt yet been made
                    if type(cut1) == tuple:
                        cutout = (list(set(cut1[0] + cut2)),resnames[i] + resnames[closest_cutout_i], (centers[i], centers[closest_cutout_i]), cut1[1])
                        done_pairs.append(pair_i)
                        redunant_merged_is.append(closest_cutout_i)
                    elif type(cut2) == tuple:
                        cutout = (list(set(cut1 + cut2[0])),resnames[i] + resnames[closest_cutout_i], (centers[i], centers[closest_cutout_i]), cut2[1])
                        done_pairs.append(pair_i)
                        redunant_merged_is.append(closest_cutout_i)
                    else:
                        cutout = (list(set(cut1 + cut2)),resnames[i] + resnames[closest_cutout_i], (centers[i], centers[closest_cutout_i]))
                        done_pairs.append(pair_i)
                        redunant_merged_is.append(closest_cutout_i)

                else: #null
                    cutout = None

        else: #solo cutout
            cut=cuts[i]
            if type(cut)==tuple:
                cutout = (cut[0], centers[i], cut[1])
            else:
                cutout = (cut, centers[i])

        cutouts.append(cutout)
    
    return cutouts,redunant_merged_is #merged or solo

def get_cutout(dask_df, distance_cutoff): #"PARENT" FUNCTION
    """for each protein in dask_df (the entire PYPKA database), it iterates residue wise through the 121,294 proteins in PYPKA database and downloads
    the structure from RCSB with biopython. Then, it checks and skips the structure if metals & hetero sulfurs are present, and deletes non-sulfur
    salts from titratable residues.
    Then, for each structure residue represented in PYPKA, generates a cutout for each residue, appends the structure to cutouts_apdb"""
    all_fnames, all_cuts, all_centers = [],[],[]
     #---PROTEINWISE RESOLUION  
    for i in range(19,20): #will equal len of set of pdbs in pypka, == 121294 
        cutouts_apdb, fnames, cutouts_1_datapoint, counter, pdbname, newfnames, centers_apdb = [],[], [],0, pdbs[i],[],[]
        residues, site_dict=[],{}
        
        Structure = pdb_parser.get_structure("",  PDBList().retrieve_pdb_file(str.lower(pdbname),obsolete=False, pdir='PDB',file_format = 'pdb'))
        structure = check_atoms_protein(Structure, Structure.get_atoms()) #returns nothing if undesirables

        if not structure: #skip entire pdb and all its entries in pypka db if there are undesirables in pdb
            continue

        ns = PDB.NeighborSearch(list(structure.get_atoms())) #set up ns , entire protein
        pdb_df = dask_df[dask_df.iloc[:, 1] == pdbname].drop(columns = ["PDB ID", "pKa"]) #make a subdf containing only residue entries which are in PYPKA (dask_df) 
        
        #---DATAPOINT RESOLUTION 
        for j in range(len(pdb_df)):  #go through each residue in a pdb #each j is a datapoint!
            NTRresname, CTRresname=None,None
            chain, res_id =pdb_df.iloc[j]['Chain'], int(pdb_df.iloc[j]['Res ID'])
            try: 
                residue=structure[0][chain][res_id] #a datapoint # will error here if pypka error 
                pypka_resname, PDBresname = pdb_df.iloc[j]['Res Name'], residue.get_resname() #pypka error

                if pypka_resname=='NTR':
                    resname, pypka_resname, NTRresname=0, "N", PDBresname

                elif pypka_resname=='CTR':
                    resname,pypka_resname, CTRresname =1,"X", PDBresname #carboxyl

                elif pypka_resname==PDBresname: 
                    resname=pypka_resname[0] #A,T,G...

                else: 
                    continue #if pypka_resname DNE PDB resname (pypka error #1)
                
                #---CUTOUT + FNAME GENERATION----#
                cutouts_1_datapoint=generate_cutout_around_protonatable_site(residue, distance_cutoff, ns, counter, resname) ##############can be multiple #returns empty if disordered
                residues.append(residue)
                cutouts_apdb.append(*cutouts_1_datapoint) 
                id = f"{pdbname}{chain}{res_id}"
                site_dict.update({id: residue})
                print(id)

                if resname=="H": 
                    fnames.append(f"{id}_{resname}{counter + .1}") 
                    fnames.append(f"{id}_{resname}{counter + .2}") 
                 
                elif CTRresname:
                    fnames.append(f"{id}_{pypka_resname}{counter}-{CTRresname}")
                elif NTRresname:
                    fnames.append(f"{id}_{pypka_resname}{counter}-{NTRresname}") 

                else: #normal
                    fnames.append(f"{id}_{resname}{counter}") 

                counter+=1

            except Exception as e: #TODO: remove this when done debugging
                print(f"Exception caught: {e}")
                raise  
  
        #os.remove(f"{local_folder}/PDB/pdb{pdbname}.ent")  #TODO
            
        #--PROTEIN WISE. MERGE OR NOT
        merged_and_solos, greaterN_pair_i = merge_or_not_cutouts(cutouts_apdb, distance_cutoff) #############
        for cut, fname in zip(merged_and_solos, fnames): #(list(set(cut1 + cut2)),resnames[i] + resnames[closest_cutout_i], (centers[i], centers[closest_cutout_i]))
            
            if cut:
                
                #resi_atom, L = site_dict[fname.split("_")[0]]['CA'], len(cut) #cntrl
                L=len(cut)

                if L==3: 
                    third = cut[2]
                    if type(third) != tuple: ##solo ter, done
                        for a in third.get_atoms(): #AMIGHT Delete
                            Fname=f"{fname}-{str(round(a.get_coord()[0],3))}"
                            break
                        #centers_apdb.append(cut[1])#
                        newfnames.append(Fname)
                        
                    #info = ((f, site_dict[f[0]]['CA'].get_coord(), protonatable_sites2[f[1][0]]) for f in (f1, f2)) #coords of CA ##?
                        save_amber_chainify_recut_deprotonate((Fname, site_dict[f[0]]['CA'].get_coord(), None), None)) #SOLO TER
                    
                    #maybe here i can feed the info which identifies, but i need to label all else so that they are not recognized. then would just need
                                                      #to change the name in resdict.
                                            
                    else: #merged normal  #done
                        f1, f2, key = fname.split("_"), friend_f.split("_"), cut[1].strip(",") #199lA162_L39_199lA159_A36_LA
                        #centers_apdb.append(third)
                        #friend_f = fnames[greaterN_pair_i[0]]
                        Fname="".join([fname,'_',fnames[greaterN_pair_i[0]],"_",key])
                        del greaterN_pair_i[0]
                        #(fname(site_dict[fname.split("_")[0]]['CA']
                        atoms_to_structure(key, Fname) #save as pdb) #cut #TODO!!!!!!!!! need to change resnames of
                        newfnames.append(Fname)
                        #info = ((Fname, site_dict[f[0]]['CA'].get_coord(), protonatable_sites2[f[1][0]]) for f in (f1, f2)) #coords of CA
                        save_amber_chainify_recut_deprotonate((Fname, (site_dict[f[0]]['CA'].get_coord(), protonatable_sites2[f[1][0]]) for f in (f1, f2)), key, flag=1)


                elif L==4: #merged, ter #TODO
                    key=cut[1]
                    for a in cut[3].get_atoms(): 
                        friend_f = fnames[greaterN_pair_i[0]]
                        Fname="".join([fname,'_',friend_f,"_",cut[1].strip(","),"-",a.get_name(), str(round(a.get_coord()[0],3))])
                        newfnames.append(Fname)
                        break
                    del greaterN_pair_i[0]
                    #centers_apdb.append(cut[2])
                    atoms_to_structure(key, Fname)
                    #info = ((f, site_dict[f[0]]['CA'].get_coord(), protonatable_sites2[f[1][0]]) for f in (f1, f2)) #coords of CA ##?
                    save_amber_chainify_recut_deprotonate(((f, site_dict[f[0]]['CA'].get_coord(), protonatable_sites2[f[1][0]]) for f in (f1, f2)), key, flag=0)
            

                else:  #solo, noemal, done
                    #centers_apdb.append(cut[1])
                    newfnames.append(fname)
                    atoms_to_structure(key, fname) 
                    #info = ((f, site_dict[f[0]]['CA'].get_coord(), protonatable_sites2[f[1][0]])) #coords of CA 
                    save_amber_chainify_recut_deprotonate(((f, site_dict[f[0]]['CA'].get_coord(), protonatable_sites2[f[1][0]])), cut[0], flag=None)

        #amber(f)
        #add_chains(f)
        #deprotonate/recut(f, site_dict[fname])
    return newfnames, centers_apdb, residues

def _ns_fXYZx2(info):
    """per residue. """
    f=info[0]
    pdbatoms = list(pdb_parser.get_structure("",  f'/Users/jessihoernschemeyer/pKaSchNet/prot/chained/{f}').get_atoms())
    ns = PDB.NeighborSearch(list(pdb_parser.get_structure("",  f'/Users/jessihoernschemeyer/pKaSchNet/prot/chained/{f}').get_atoms())) #set up ns , entire protein
    #cut = ns.search(centerA[0], distance_cutoff, "A"), ns.search(centerB[1], distance_cutoff, "A")
    return f, info[1], info[2], ns, ns.search(centerA[0], distance_cutoff, "A"), ns.search(centerB[1], distance_cutoff, "A")
    
def amber(fname):
    skript = f"""source leaprc.protein.ff14SB
    source leaprc.water.tip3p
    loadOff "/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/dat/leap/lib/amino19.lib"
    mol = loadpdb "/Users/jessihoernschemeyer/pKaSchNet/cuts/{fname}.pdb"
    savepdb mol "/Users/jessihoernschemeyer/pKaSchNet/prot/{fname}"

    quit"""
    with open("ascript.py","w") as file: 
        file.writelines(skript)
    
def add_chains(file):
    chain="A"
    counter=0
    with open(local_folder + f'/prot/{file}', 'r') as f, open(local_folder + f'/prot/chained/{file}', 'w') as out:
        for line in f:
            if line.startswith("ATOM"):
                # Extract the fields based on PDB format fixed columns
                serial = line[6:11].strip()
                atom_name = line[12:16].strip()
                residue_name = line[17:20].strip()
                res_seq = line[22:26].strip()
                x = line[30:38].strip()
                y = line[38:46].strip()
                z = line[46:54].strip()
                out.write(f"ATOM  {int(serial):5d}  {atom_name:<4}{residue_name:>3} {chain:1}{int(res_seq):4d}    {float(x):8.3f}{float(y):8.3f}{float(z):8.3f}  1.00  0.00\n")

            elif line.startswith("TER"):
                counter += 1
                chain = char_list[counter]

def deprot_save_solo_reg(info, key): #done
    "not his!"
    f, coord, center, cut =  _ns_fXYZx2(info)
    #cut=ns.search(center, distance_cutoff, "A")
    #save(cut, f + "P")

    for i in range(len(cut)):
        a=cut[i]
        if a.get_name() == protonatable_sites[key]:
            papa = a.get_parent()
            if coord == papa['CA'].get_coord():#ctrl
                break

    del cut[i]
    save(cut, f + "d")

def deprot_save_solo_ter(info):   #TODO!!     
            cut=ns.search(center, distance_cutoff, "A")
            save(cut, f + "P")
        
            res=f.split("_")[1].split("-")
            atom = protonatable_sites[res[0][0]] #X,N

            if type(atom) == str: #NTR
                for i in range(len(cut)):
                    if a.get_name() == atom:
                        papa = a.get_parent()
                        if info_resdict[papa.get_resname()] == papa['CA'].get_coord():#ctrl
                            break
            
            
            else: #tuple, C-ter
                for i in range(len(cut)):
                    a=cut[i]
                    if a.get_name() in atom:
                        papa = a.get_parent()
                        if info_resdict[papa.get_resname()] == papa['CA'].get_coord():#ctrl
                            break

                del cut[i]
                save(cut, f + "d")
                
            for i in range(len(cut)):
                a=merged[i]
                if a.get_name() == protonatable_sites[key]:
                    papa = a.get_parent()
                    if info_resdict[papa.get_resname()] == papa['CA'].get_coord():#ctrl
                        break

                del cut[i]
                save(cut, f + "d")

def deprot_save_merged_ter(info, key): #mostly done
    "cant do AA! or his"
    atoms_i=[]

    res1,res2=info[0],info[1] #tuple, tuple ((f, coord, resname,
    f, coordA, centerA,cut1 = _ns_fXYZx2(info[0])
    f, coordB, centerB,cut2 = _ns_fXYZx2(info[1])
    info_resdict={res1[2] : res1[1], res2[2] : res2[1]}

    f_split=f.split("_")
    resA, resB = f_split[1].split("-"), f_split[3].split("-") #N0-MET, G1 --> [N0, MET], [G1]

    
    counter=0

    #recut
    merged = set(cut1 + cut2) 
    save(merged, f + "P")

    if len(resA) > 1: #res A is TER [N0, MET]
        ter_res = resA[1] #MET
        reg_res_key = resB[0]
        reg_res = protonatable_sites2[reg_res_key] #GLU #G=resB[0]

    if len(resB) > 1: #res B is TER
        ter_res = resB[1]
        reg_res_key = resA[0]
        reg_res =protonatable_sites2[reg_res_key]
 
   
    for i in range(len(merged)):
        a=merged[i]
        papa = a.get_parent()
        resname = papa.get_resname()
        if resname == ter_res: 
            if info_resdict[resname] == papa['CA'].get_coord():
                if papa.has_id('OXT'):
                    atom = papa["OXT"]
                    atoms_i.append({merged.index(atom):ter_res})
                    #atoms_to_delete.append({papa: "OXT"})
                    counter+=1
                    if counter == 2:
                            break      

                else:
                    atom = papa["C"]
                    atoms_i.append({merged.index(atom):ter_res})
                    #atoms_to_delete.append({papa: "C"})
                    counter+=1
                    if counter == 2:
                            break      

        elif resname == reg_res:
            if info_resdict[resname] == papa['CA'].get_coord():
                atom = papa[protonatable_sites[reg_res_key]]
                atoms_i.append({merged.index(atom):reg_res})
                #atoms_to_delete.append({papa: protonatable_sites[reg_res_key]})
                counter+=1
                if counter == 2:
                        break      

    Bd, A, B = merged.copy(), atoms_i[0], atoms_i[1]
    #ApBp
    save(merged, f + "P")

    #deprotonate A
    del merged[A] #delete first atom from ApBp
    save(merged, f + A.value()) #ApBp

    #deprotonate B          
    del Bd[B]
    save(Bd, f + B.value())

    #AdBd
    save(list({merged} ^ + {Bd}), f + "D") #symmetrical difference = AdBd

    return




def deprotonate_save_merged(info, key): #done
    "((info1), {info2)). f is the same for both! cant do AA! or his"
    atoms_i=[]
    res1,res2=info[0],info[1] #tuple, tuple
    f, coordA, centerA,cut1 = _ns_fXYZx2(info[0])
    f, coordB, centerB,cut2 = _ns_fXYZx2(info[1])

    info_resdict={res1[2] : res1[1], res2[2] : res2[1]} #name of res, coord of res {ASP:C1, ASP:C2}
    counter=0
    #cut1, cut2 = ns.search(centerA[0], distance_cutoff, "A"), ns.search(centerB[1], distance_cutoff, "A")

    #recut
    merged = set(cut1 + cut2) 

    #DEPROTONATE

    #acquire atoms to delete
    atoms = (protonatable_sites[k] for k in key) #Hs names
    for i in range(len(merged)): #iterate thru atoms
        a=merged[i]#atom in merged
        if a.get_name() in atoms: #if atomname == hydrogen atom of res name
            papa = a.get_parent()
            resname=papa.get_resname()
            if info_resdict[resname] == papa['CA'].get_coord():#CA coord of Tres of OG cut == Tres's alphacarbon position, which would be in the cut. #TODO: if one were to use reallu small distances this may not work :)
                atoms_i.append({i:resname[0]}) #{3 : 'A', 12 : 'A'}
                counter+=1
                if counter == 2: #if it found 2 site,leave #his wouldnt work here
                    break
    Bd, A, B = merged.copy(), atoms_i[0], atoms_i[1]
    
    #ApBp
    save(merged, f + "P")

    #deprotonate A
    #A=atoms_i[0] #first atom's index
    del merged[A] #delete first atom from ApBp
    save(merged, f + A.value()) #ApBp

    #deprotonate B          
    #B=atoms_i[1]
    del Bd[B]
    save(Bd, f + B.value())

    #AdBd
    save(list({merged} ^ + {Bd}), f + "D") #symmetrical difference = AdBd

    return

def save_amber_chainify_recut_deprotonate(info, key, flag):
        """info (tuple or tuple of tuples): ((f, alpha carbon of the titratable residue, resname, cut))
        #TODO: fix error in SAVE!
        """
        #atoms_i=[]
        #f, coord, center =info[0], info[1], info[2]
        #pdbatoms = list(pdb_parser.get_structure("",  f'/Users/jessihoernschemeyer/pKaSchNet/prot/chained/{f}').get_atoms())
        #ns = PDB.NeighborSearch(pdbatoms) #set up ns , entire protein
        f=info[0][0] #is the same for both
        #protonate
        amber(f)

        #add chains to protonated pdb
        add_chains(f)

        #info_resdict={res1[2] : res1[1], res2[2] : res2[1]} #name of res, coord of res
        #MERGED
        if flag==1: 
            deprotonate_save_merged(info, key)


        #ter merged
        elif flag==0:
            cut1=ns.search(center[0], distance_cutoff, "A")
            cut2=ns.search(center[1], distance_cutoff, "A")
            f_split=f.split()
            resA, resB = f_split[1].split("-"), f_split[3].split("-")

            if len(resA) > 1:
                res1 = resA[1]
                res2 = protonatable_sites[resB[0]]

            if len(resB) > 1:
                res2 = resB[1]
                res1=protonatable_sites[resA[0]]
            
            merged = set(cut1 + cut2)
            save(merged, f + "P")
            
            for i in range(len(merged)):
                a=merged[i]
                if a.get_name() in atoms_to_delete:
                    papa = a.get_parent()
                    if papa.get_resname() in (res1, res2):
                        if rs['CA'].get_coord() == papa['CA'].get_coord():
                            atoms_i.append(i)
                            counter+=1
                            if counter == 2:
                                break
                                
            Bd = merged.copy()

            del merged[atoms_del[0]]
            save(cut, f + "Ad")

            del Bd[atoms_del[1]]
            save(Bd, f + "Bd")

            save(set(merged + Bd), f + "D")


        #solo
        else: #none flag

            if key != None: #solo, noemal
                deprot_save_solo_reg(info, key) 

            else: #solo, ter
                deprot_save_solo_ter(info)  #TODO

            
#dask_df = read_database(pkPDB_CSV)
#pdb_parser, pdbs = PDB.PDBParser(), list(OrderedSet(list(dask_df["PDB ID"])))
fs, cs, rs = get_cutout(dask_df, 5)


consider putting cA in fname

fix REDUNDANT MERGED IS!!! (still?)

HIS, change names of 

In [5]:
%%capture
                

"""for f in fs:
    amber(f)
    #!tleap -s -f /Users/jessihoernschemeyer/pKaSchNet/ascript.py
    add_chains(f)"""

In [ ]:


def deprotonate_singles(res, cut): #turn one acidic into all its others? #NOT HIS!!!
    """Binary Situation
    this function takes a cutout that is not merged / a single cutout, and the file name of that cutout. It gets the residue string name, and from that a string of the atom name of the hydrogen ion
    that should be deleted. 
    cut: Set(<atom1>, <atom2>)
    ONE CCUT AT A TIME!
    Then, found atoms is a list of all the atoms with that name e.g. "OXT" and the desired parent res (in the file name).
    If the atom has the name and parent res name matching the file name information, then the atom is detached and then saved.
    
    This returns: ApBd or AdBp."""
#NORMAL CASE
    atom_to_delete=None

    atom_to_delete = protonatable_sites.get(res)
    if atom_to_delete:
        Res = protonatable_sites2[res] #careful here w same name

        for atom in cut:
            residue=atom.get_parent()
            if residue.get_resname() == Res:
                residue.detach_child(atom_to_delete)
                return cut
            
    if not atom_to_delete: #his (atom), or TER
        if res[0]=="+": #ter
            for atom in cut:
                residue=atom.get_parent()
                if residue.get_resname() == "HIP":
                    residue.detach_child(res) #res IS atom to delete for HIS!
                    return cut
        else: #his
            for atom in cut:
                residue=atom.get_parent()
                if residue.get_resname() == "HIP":
                    residue.detach_child(res) #res IS atom to delete for HIS

            deprotonate_terminus_single(res, cut)

def deprotonate_terminus_single(res, cut):
    """Input: XPHE, NARG..
    returns a deprotonated NTR or CTR"""
    ter, resi = res[0], res[1:]
    for atom in cut:
        residue=atom.get_parent()
        if residue.get_resname() == resi: 
            if res=="X": #CTR
                atoms = residue.get_atoms()
                try: 
                    residue.detach_child("OXT")
                    return cut 
                except:
                    residue.detach_child("C")
                    return cut
            else: #NTR 
                residue.detach_child("N")
                return cut

def dp_GleichRes2Mal(res, cut): #doesnt take HHH or ters
    "returns two cuts, deprotonated of them both."
    
    found_atoms, dp_cuts =[],[]
    atom_to_delete = protonatable_sites[res]

    if type(atom_to_delete) == tuple: #histidine
        for atom in cut:
            residue=atom.get_parent()
            if residue.get_resname() == "HIP":
                for atom in residue.get_atoms():   
                    atomname = atom.get_name()        
                    if atomname in atom_to_delete:
                        found_atoms.append((atomname, residue))

        
        found_atoms.sort() #will sort by letter and number 
        resA_del, resB_del, resA_eps, resB_eps = found_atoms[0], found_atoms[1], found_atoms[2], found_atoms[3]
        
        hipAhipB = cut.copy() 
        dp_cuts.append(hipAhipB, "HIPHIP")
        #modify resA
        resA_del[1].detach_child(resA_del[0]) #makes HIE res A, res B HIP
        hieAhipB = cut.copy()
        dp_cuts.append(hieAhipB, "HIEHIP") #HIE+HIp

        resA_eps[1].detach_child(resA_eps[0]) #makes HIS res A, res B HIP
        hisAhipB = cut.copy()
        dp_cuts.append(hisAhipB, "HISHIP") #HIS+HIp

        hidAhipB = hisAhipB.union(hipAhipB-hieAhipB)
        dp_cuts.append(hidAhipB, "HIDHIP")

        #modify residue b 
        resB_del[1].detach_child(resB_del[0]) #makes HIE res A, res B HIP
        hisAhieB = cut.copy()
        dp_cuts.append(hisAhieB, "HISHIE")

        resB_eps[1].detach_child(resB_eps[0])
        hisAhisB = cut.copy()
        dp_cuts.append(hisAhisB, "HISHIE")

        hisAhidB = hisAhisB.union(hisAhipB-hisAhieB)
        dp_cuts.append(hisAhisB, "HISHID")
        
        #now make the rest 
        hipA_atoms = hipAhipB - hisAhipB 
        HIPHIE = hisAhieB.union(hipA_atoms) #arg is the atoms which make hip seperate from his
        dp_cuts.append(HIPHIE, "HIPHIE")
        HIPHID = hisAhidB.union(hipA_atoms)
        dp_cuts.append(HIPHID, "HIPHID")
        HIPHIS = hisAhisB.union(hipA_atoms)
        dp_cuts.append(HIPHIS, "HIPHIS")

        dp_cuts.append(HIPHIE | hieAhipB, "HIEHIE")
        dp_cuts.append(HIPHID | hieAhipB, "HIEHID")
        dp_cuts.apprnf(HIPHIS | hieAhipB,  "HIEHIS")

        dp_cuts.append(HIPHIE | hidAhipB, "HIDHIE") 
        dp_cuts.append(HIPHID | hidAhipB, "HIDHID")
        dp_cuts.append(HIPHIS | hidAhipB, "HIDHIS")

    elif len(atom_to_delete) == 2: #nor his
        Res = protonatable_sites2[res]
        for residue in cut.get_residues():
            if residue.get_name() == Res:
                for atom in residue.get_atoms():           
                    if atom.get_name() == atom_to_delete:
                        found_atoms.append((residue, atom_to_delete))

        #for two found atoms
        resA, resB = found_atoms[0], found_atoms[1]
        resA[0].detach_child(resA[1])
        Adp = cut.copy()
        #deprotonate cut fully by removing the other second residue
        resB[0].detach_child(resB[1])
        Bdp = cut.copy()

        return Adp, resB

def recut_and_deprotonate(fnames_apdb, centers_apdb, distance_cutoff): #the centers come in #fnames after protonation
    """
    fnames_apdb [list]: [fname_cut1, fname_cut2, fname_cut3] --> resA &/OR resB
                    Ex: ['199lA11_GLU3', '199lA10_ASP2_199lA161_TYR37_AT', '199lA70_ASP22_199lA31_HIS11.2_AH'] 

    This function takes in all the file names for a single pdb, as well as the centers of their titratable site (OXT, N, NE2, NH...). Using the fname, it
    makes a cutout again using that center which was found before. 
    
    If the cutout is merged, then there will be a multiple centers. Then, the cutout becomes the set of both cutoute (a sphere of distance_cutoff from the 
    protonatable site.)"""
    for fname, center in zip(fnames_apdb, centers_apdb):
        struct = pdb_parser.get_structure("",  f'/Users/jessihoernschemeyer/pKaSchNet/prot/chained/{fname}')
        pdbatoms = list(struct.get_atoms())
        ns = PDB.NeighborSearch(pdbatoms) #set up ns , entire protein
        print("all atoms", len(list(struct.get_atoms())))
        #############MERGEDD#####
        if type(center)==tuple: 
            f=fname.split("_")
            print(f)
            singles_res, key=f[1],f[4] #22X-PHE, AT, XASP
            resA, resB = key[0],key[1] #A,T

            #NTR/CTR (cannot combine NTR and CTR.) MERGED
            if resA or resB in ('X', 'N'): #need to deal wih his in here???
                cut = set(ns.search(center[0], distance_cutoff, "A")) | set(ns.search(center[1], distance_cutoff, "A")) #the merged cut
                Save(cut, fname) #fully protonated
                if resA in ('X', 'N'): 
                    #res1 = f[1].split("-") #the real residue
                    resAdp = deprotonate_singles(resA + f[1].split("-")[1], cut) #xphe
                    resBdp = deprotonate_singles(resB, cut) #xphe

                elif resB in ('X', 'N'):
                    res2 = f[3].split("-")
                    resBdp = deprotonate_singles(resB + res2[1], cut) #xphe
                    resAdp = deprotonate_singles(resA)
                    
                deprotonated = OrderedSet(resAdp) | OrderedSet(resBdp)  #fully deprotonated #not making a new cut but combining what is done 

                Save(resAdp, fname + f'~{resA}d') #resA deprot, B prot
                Save(resBdp, fname + f"~{resB}d") #A prot, resB deprot
                Save(deprotonated, fname + "~d")



            elif resA == resB: #no ntr and ctr will get here!
                if resB == 'H' or resA=='H':  #double H
                    his_cuts = dp_HH("H", cut)
                    for cut in his_cuts:
                        Save(cut[0], fname + f'{cut[1]}')

                else: #normal AA, GG..
                  
                    resAdp, resBdp = dp_GleichRes2Mal(cut, resA)   ##AA, GG... #res A is res B
                    Save(resAdp, fname + f'~{resA}d') #resA deprot, B prot
                    Save(resBdp, fname + f"~{resB}d") #A prot, resB deprot
                    Save(OrderedSet(resBdp) | OrderedSet(resAdp), fname + '~d')
                
            elif resB == 'H' or resA=='H': #histidine 
                    if resA == 'H': #1 his #HA
                        HIE = deprotonate_singles("HD1", cut)
                        Save(HIE, fname + '~HIE') #A, eps prot with B prot
                        HID = deprotonate_singles("HE2", cut)
                        Save(HID, fname + '~HID') #A, delta protonated
                        HIS = deprotonate_singles("HD1", HID.copy()) 
                        Save(HIS, fname + '~HIS') #A fully deprotonated

                        resBdp = deprotonate_singles(resB, cut.copy())

                        #remaining combos with deprotonated other nonhis and nonter residue
                        resBd_HIE = OrderedSet(resBdp) | OrderedSet(HIE) 
                        Save(resAd_HIE, fname + f'~{resB}d+HIE')
                        resBd_HID = OrderedSet(resBdp) | OrderedSet(HID)
                        Save(resAd_HID, fname + f'~{resB}d+HID')
                        resBd_HIS = OrderedSet(resBdp) | OrderedSet(HIS)
                        Save(resAd_HIS, fname + f'~{resB}d+HIS')
                
                    else: #ResB = H. then this means that the second res is the his. AH
                        HIE = deprotonate_singles("HD1", cut.copy())
                        Save(HIE, fname + '~HIE') #HIE with A =HIP
                        HID = deprotonate_singles("HE2", cut.copy())
                        Save(HID, fname + '~HID')
                        HIS = deprotonate_singles("HD1", HID.copy())
                        Save(HID, fname + '~HIS') #fully deprotonated

                        resAdp = deprotonate_singles(resA, cut.copy())

                        #make combos with sets 
                        resAd_HIE = OrderedSet(resAdp) | OrderedSet(HIE) #after resA is already deprotonated
                        Save(resAd_HIE, fname + f'~{resA}d+HIE')
                        resAd_HID = OrderedSet(resAdp) | OrderedSet(HID)
                        Save(resAd_HID, fname + f'~{resA}d+HID')
                        resAd_HIS = OrderedSet(resAdp) | OrderedSet(HIS)
                        Save(resAd_HIS, fname + f'~{resA}d+HIS')
                        
            else:  #normal merged
                    cut = OrderedSet(ns.search(center[0], distance_cutoff, "A")) | OrderedSet(ns.search(center[1], distance_cutoff, "A")) #the merged cut
                    print(len(cut))
                    list(set(cut1[0] + cut2))
                    Save(cut, fname) #fully protonated, both

                    resAdp = deprotonate_singles(resA, cut) #resA = "A", "G". resAdp = a cut
                    Save(resAdp, fname + f'~{resA}d') #resA deprot, B prot
                    print(len(resAdp))

                    resBdp = deprotonate_singles(resB, cut)
                    Save(resBdp, fname + f"~{resB}d") #A prot, resB deprot

                    deprotonated = OrderedSet(resAdp) | OrderedSet(resBdp)  #fully deprotonated #not making a new cut but combining what is done 
                    Save(deprotonated, fname + "~d")

        else: #single
            cut = ns.search(center, distance_cutoff, "A")
            Save(cut, fname) #fully protonated

            L = singles_res.split("-")
            if len(L)==2: #TER
                 #debug
                 for atom in cut.get_atoms():
                     print(atom)
                 deprotonated = deprotonate_singles(L[1], cut) #L[1] = "PHE"
                 Save(deprotonated, fname + "~D")

            else: 
                if len(L[0].split(".")) == 2: #HIS
                    HIP=cut.copy()

                    HIE = deprotonate_singles("OD1", cut)
                    Save(HIE, fname + "HIE")

                    HIS = deprotonate_singles("OE2", HIE.copy())
                    Save(HIS, fname + "HIS")
                    
                    Save(HIS.union(HIP - HIE), fname + "HID")
                    
                else: #regular
                    deprotonated = deprotonate_singles(fname.split("_")[1][0], cut) #[A]SP22, [X]25PHE.. #not his
                    Save(deprotonated, fname + "~D")

def Save(cut, F):
    if cut:
        print(F, len(cut))
    else:
        print(F, cut)
          
recut_and_deprotonate(fs, cs, 5)

In [65]:
cut1=ns.search(center[0], 5, "A")
cut, resis=cut1,["ASH"]
atoms_to_delete=["OD1", "OD2"]
#save(cut, "P")
#merged, normal
for i in range(len(cut)):
    a=cut[i]
    if a.get_name() in atoms_to_delete:
        if a.get_parent().get_resname() in resis:
            

atoms_del = [i for i in range(len(cut)) if cut[i].get_name() in atoms_to_delete and cut[i].get_parent().get_resname() == res]
print(atoms_del, cut[atoms_del[0]].get_parent(), cut[atoms_del[1]].get_parent(), cut[atoms_del[2]].get_parent())
Bd = cut.copy()

del cut[atoms_del[0]]
#save(cut, "Ad")
del Bd[atoms_del[1]]
#save(Bd, "Bd")
#save(set(cut, Bd), "D")



IndexError: list index out of range

In [ ]:
res_cut = ns.search(center[0], distance_cutoff, "R") #list

for r in res_cut:
    if r.get_resname() == res:
        r.detach_child(atom_to_delete)
        break
    return [tuple(residue.get_atoms()) for residue in res.get_cut] 



In [55]:
merged=1
center=cs[1]
fname=fs[1]

struct = pdb_parser.get_structure("",  f'/Users/jessihoernschemeyer/pKaSchNet/prot/chained/{fname}')
pdbatoms = list(struct.get_atoms())
ns = PDB.NeighborSearch(pdbatoms) #set up ns , entire protein

if merged:
    cut1=ns.search(center[0], 5, "A")
    cut2=ns.search(center[1], 5, "A")
print(len(list(struct.get_atoms())), len(cut1), len(cut2), len(set(cut1 + cut2)), len(set(cut1)  | set(cut2)))

for a in cut1:
    a.get_parent().detach_child('N')
    break
print(len(list(struct.get_atoms())), len(cut1), len(cut2), len(set(cut1 + cut2)), len(set(cut1)  | set(cut2)))
    
    

151 57 54 82 82
150 57 54 82 82


/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'H' for Atom (name=H1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'H' for Atom (name=H2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'H' for Atom (name=H3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/jessihoernschemeyer/miniconda3/envs/cfcnn/lib/python3.9/site-packages/Bio/PDB/Atom.py:232: PDB

todo disordered

In [ ]:
%%capture
local_folder="/Users/jessihoernschemeyer/pKaSchNet"
from Bio.PDB import *
from Bio import PDB
pdb_parser=PDB.PDBParser()
cut = pdb_parser.get_structure("",  local_folder + '/prot/chained/199lA161_T37_199lA10_A2_TA')

for residue in cut.get_residues():
    for a in residue.get_atoms():
        info = f"{a.get_name()}{round(a.get_coord()[0],3)}"
        

cut = pdb_parser.get_structure("",  local_folder + '/cuts/199lA161_T37_199lA10_A2_TA')

for residue in cut.get_residues():
    for a in residue.get_atoms():
        info = f"{a.get_name()}{round(a.get_coord()[0],3)}"


In [24]:
cut[0]['A'][161]

<Residue TYR het=  resseq=161 icode= >

In [19]:
print(a.get_full_id())

('', 0, 'C', (' ', 10, ' '), ('H2', ' '))


In [1]:
fs

NameError: name 'fs' is not defined